# Solutions — 04 GMM Tests & Diagnostics

In [ ]:
import sys
import warnings
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

project_root = Path("../../..").resolve()
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from panelbox.gmm import DifferenceGMM, SystemGMM, GMMOverfitDiagnostic

warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=RuntimeWarning)
print("Setup complete.")

---
## Exercise 1: Diagnose Problems

Load `bad_specification.csv`, estimate with omitted variable, diagnose, then fix.

In [ ]:
# 1a. Estimate with omitted variable
df_bad = pd.read_csv('../data/bad_specification.csv')

model_bad = DifferenceGMM(
    data=df_bad, dep_var='y', lags=1,
    exog_vars=['x1'],  # Missing x2_omitted!
    id_var='entity', time_var='time',
    collapse=True, two_step=True, robust=True,
    time_dummies=False  # Avoids under-identification with collapse=True
)
results_bad = model_bad.fit()

print("=== Misspecified Model ===")
print(f"L1.y coef: {results_bad.params.iloc[0]:.4f}")
print(f"Hansen J: stat={results_bad.hansen_j.statistic:.4f}, p={results_bad.hansen_j.pvalue:.4f}")
print(f"AR(2): p={results_bad.ar2_test.pvalue:.4f}")
print(f"\nHansen J conclusion: {results_bad.hansen_j.conclusion}")

In [ ]:
# 1b. Run overfit diagnostic
diag_bad = GMMOverfitDiagnostic(model_bad, results_bad)
bounds_bad = diag_bad.coefficient_bounds_test()
print("Bounds test (misspecified):")
print(f"  OLS={bounds_bad['ols_coef']:.4f}, FE={bounds_bad['fe_coef']:.4f}, "
      f"GMM={bounds_bad['gmm_coef']:.4f}")
print(f"  Signal: [{bounds_bad['signal']}]")

In [ ]:
# 1c. Fix: add omitted variable
model_fix = DifferenceGMM(
    data=df_bad, dep_var='y', lags=1,
    exog_vars=['x1', 'x2_omitted'],  # Now complete
    id_var='entity', time_var='time',
    collapse=True, two_step=True, robust=True,
    time_dummies=False
)
results_fix = model_fix.fit()

print("=== Fixed Model ===")
print(f"L1.y coef: {results_fix.params.iloc[0]:.4f}")
print(f"Hansen J: stat={results_fix.hansen_j.statistic:.4f}, p={results_fix.hansen_j.pvalue:.4f}")
print(f"AR(2): p={results_fix.ar2_test.pvalue:.4f}")

# Verify fix
diag_fix = GMMOverfitDiagnostic(model_fix, results_fix)
bounds_fix = diag_fix.coefficient_bounds_test()
print(f"\nBounds: [{bounds_fix['signal']}] {bounds_fix['details']}")

---
## Exercise 2: AR(2) Investigation

Test how AR(2) p-values change with different `phi2` values.

In [ ]:
def simulate_ar2_errors(N=200, T=7, rho=0.5, phi2=0.3, seed=42):
    """Simulate panel data with AR(2) errors."""
    np.random.seed(seed)
    data = []
    for i in range(N):
        mu_i = np.random.normal(0, 1)
        y = [mu_i + np.random.normal(0, 1)]
        eps = [np.random.normal(0, 1)]
        for t in range(1, T):
            eps_t = (phi2 * eps[t-2] if t >= 2 else 0) + np.random.normal(0, 1)
            eps.append(eps_t)
            y.append(rho * y[-1] + mu_i + eps_t)
        for t in range(T):
            data.append({'id': i, 'time': t, 'y': y[t]})
    return pd.DataFrame(data)

# Test with different phi2 values
phi2_values = [0.0, 0.1, 0.3, 0.5]
results_list = []

for phi2 in phi2_values:
    df_sim = simulate_ar2_errors(phi2=phi2)
    model = DifferenceGMM(
        data=df_sim, dep_var='y', lags=1,
        id_var='id', time_var='time',
        collapse=True, two_step=True, robust=True
    )
    res = model.fit()
    results_list.append({
        'phi2': phi2,
        'ar1_pval': res.ar1_test.pvalue,
        'ar2_pval': res.ar2_test.pvalue,
        'rho_hat': res.params.iloc[0],
        'hansen_p': res.hansen_j.pvalue
    })

df_results = pd.DataFrame(results_list)
print(df_results.to_string(index=False))
print("\nConclusion: As phi2 increases, AR(2) p-value drops. ")
print("At phi2=0.3, AR(2) typically rejects at 10% level.")

---
## Exercise 3: Overfitting Diagnostic

Compare `GMMOverfitDiagnostic` for Difference and System GMM on growth data.

In [ ]:
df_growth = pd.read_csv('../data/growth.csv')

# Difference GMM (time_dummies=False: T=20 would create too many dummies)
model_diff = DifferenceGMM(
    data=df_growth, dep_var='lgdp', lags=1,
    exog_vars=['inv', 'school'],
    id_var='country', time_var='year',
    collapse=True, two_step=True, robust=True,
    time_dummies=False
)
results_diff = model_diff.fit()

# System GMM
model_sys = SystemGMM(
    data=df_growth, dep_var='lgdp', lags=1,
    exog_vars=['inv', 'school'],
    id_var='country', time_var='year',
    collapse=True, two_step=True, robust=True,
    time_dummies=False,
    level_instruments={'max_lags': 1}
)
results_sys = model_sys.fit()

# Diagnostics
print("=" * 70)
print("DIFFERENCE GMM")
print("=" * 70)
diag_diff = GMMOverfitDiagnostic(model_diff, results_diff)
print(diag_diff.summary(run_jackknife=True))

print("\n\n")
print("=" * 70)
print("SYSTEM GMM")
print("=" * 70)
diag_sys = GMMOverfitDiagnostic(model_sys, results_sys)
print(diag_sys.summary(run_jackknife=True))

In [ ]:
# Compare feasibility
feas_diff = diag_diff.assess_feasibility()
feas_sys = diag_sys.assess_feasibility()

print(f"{'':25s} {'Diff GMM':>12s} {'System GMM':>12s}")
print("-" * 55)
print(f"{'Instruments':25s} {feas_diff['n_instruments']:>12d} {feas_sys['n_instruments']:>12d}")
print(f"{'Groups':25s} {feas_diff['n_groups']:>12d} {feas_sys['n_groups']:>12d}")
print(f"{'Ratio':25s} {feas_diff['instrument_ratio']:>12.3f} {feas_sys['instrument_ratio']:>12.3f}")
print(f"{'Signal':25s} {feas_diff['signal']:>12s} {feas_sys['signal']:>12s}")
print("\nSystem GMM has more instruments (both difference and level equation).")
print("If ratio exceeds 1.0, System GMM shows more overfitting risk.")

---
## Exercise 4: Complete Validation

Full validation workflow on the firm investment data.

In [ ]:
df_firm = pd.read_csv('../data/firm_investment.csv')
print(f"Firm data: {df_firm.shape[0]} obs, {df_firm['firm'].nunique()} firms")

# Difference GMM
model_d = DifferenceGMM(
    data=df_firm, dep_var='ik', lags=1,
    exog_vars=['q', 'cashflow'],
    id_var='firm', time_var='year',
    collapse=True, two_step=True, robust=True,
    time_dummies=False
)
results_d = model_d.fit()
print("\nDifference GMM:")
print(results_d.summary())

In [ ]:
# System GMM
model_s = SystemGMM(
    data=df_firm, dep_var='ik', lags=1,
    exog_vars=['q', 'cashflow'],
    id_var='firm', time_var='year',
    collapse=True, two_step=True, robust=True,
    time_dummies=False,
    level_instruments={'max_lags': 1}
)
results_s = model_s.fit()
print("System GMM:")
print(results_s.summary())

In [ ]:
# Overfit diagnostics side-by-side
print("\n" + "=" * 70)
print("DIFFERENCE GMM — OVERFIT DIAGNOSTIC")
print("=" * 70)
diag_d = GMMOverfitDiagnostic(model_d, results_d)
print(diag_d.summary(run_jackknife=False))

print("\n" + "=" * 70)
print("SYSTEM GMM — OVERFIT DIAGNOSTIC")
print("=" * 70)
diag_s = GMMOverfitDiagnostic(model_s, results_s)
print(diag_s.summary(run_jackknife=False))

In [ ]:
# Final recommendation
print("\n=== FINAL RECOMMENDATION ===")
print("\nKey checks:")
print(f"  Diff GMM — AR(2) p={results_d.ar2_test.pvalue:.4f}, "
      f"Hansen p={results_d.hansen_j.pvalue:.4f}, "
      f"ratio={results_d.instrument_ratio:.3f}")
print(f"  Sys  GMM — AR(2) p={results_s.ar2_test.pvalue:.4f}, "
      f"Hansen p={results_s.hansen_j.pvalue:.4f}, "
      f"ratio={results_s.instrument_ratio:.3f}")

if results_s.diff_hansen is not None:
    print(f"  Diff-in-Hansen p={results_s.diff_hansen.pvalue:.4f}")
    if results_s.diff_hansen.pvalue > 0.10:
        print("\n-> System GMM preferred (level instruments valid, more efficient)")
    else:
        print("\n-> Difference GMM preferred (level instruments rejected)")
else:
    print("\n-> Compare SE: use whichever is more efficient with valid diagnostics")